In [1]:
import pyscf
import pyscf.tools
from pyscf import lo, gto


from orbitalpartitioning import *


molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
 """

pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   False,
        spin    =   10, # number of unpaired electrons
        charge  =   -2)
pymol.build()

mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "minao"
mf.run()
##################################################################




******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5289.63018546547
  HOMO = 0.473246078347928  LUMO = 0.476436886667598
cycle= 1 E= -5289.15069324983  delta_E= 0.479  |g|= 2.84  |ddm|= 8.23
  HOMO = -0.000303795913073977  LUMO = 0.0865952448133458
cycle= 2 E= -5276.15095510045  delta_E=   13  |g|= 4.14  |ddm|= 9.58

WARN: HOMO 0.91237260160555 >= LUMO 0.129031757301459

cycle= 3 E= -5291.20851605662  delta_E= -15.1  |g|= 2.39  |ddm|= 9.69
  HOMO = 0.192121528469619  LUMO = 0.454224553216574
cycle= 4 E= -5297.24364203003  delta_E= -6.04  |g|= 1.78  |ddm|= 8.96
  HOMO = 0.193863066010502  LUMO = 0.42795

ROHF-RHF object of <class 'pyscf.scf.rohf.ROHF'>

# ROHF/6-31G*

In [2]:
pymol.basis = "6-31g*"
pymol.build()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "chkfile"
mf.run()
##################################################################



******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF-RHF
initial guess = chkfile
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5355.01675380936
  HOMO = 0.118460129711002  LUMO = 0.348131329113529
cycle= 1 E= -5356.63553764889  delta_E= -1.62  |g|= 0.699  |ddm|= 2.87
  HOMO = -0.105049941073764  LUMO = 0.332215864709768
cycle= 2 E= -5355.38925263732  delta_E= 1.25  |g|= 1.83  |ddm|= 1.84
  HOMO = 0.0817638485561308  LUMO = 0.343204640393592
cycle= 3 E= -5356.68964568039  delta_E= -1.3  |g|= 0.445  |ddm|= 1.66
  HOMO = 0.00718713219997945  LUMO = 0.333756751381623
cycle= 4 E= -5356.67058202987  delta_E= 0.0191  |g|= 0.339  |ddm|= 0.776
  HOMO = 0.012506899826988  LUMO = 0.333

ROHF-RHF object of <class 'pyscf.scf.rohf.ROHF'>

In [3]:
#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]

ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]

print(" Norbs = %6i ndocc = %6i nsing = %6i nvirt = %6i"%(Cdocc.shape[0], ndocc, nsing, nvirt))

 Norbs =    190 ndocc =     77 nsing =     10 nvirt =    103


In [4]:
# Find AO's corresponding to atoms
doccs = []
frags = [[] for i in range(3)]
virts = []

for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    atom_number = ao[0]
    orbital = ao[2]
    if atom_number == 0:
        if orbital in ("3s", "3p"):
            doccs.append(ao_idx)
        if orbital in ("3d",):
            frags[0].append(ao_idx)
        if orbital in ("4s", "4p", "4d"):
            virts.append(ao_idx)
    if atom_number == 1:
        if orbital in ("2s"):
            doccs.append(ao_idx)
        if orbital in ("2p","3s","3p"):
            frags[1].append(ao_idx)
        if orbital in ("3d"):
            virts.append(ao_idx)
    if atom_number == 2:
        if orbital in ("3s", "3p"):
            doccs.append(ao_idx)
        if orbital in ("3d",):
            frags[2].append(ao_idx)
        if orbital in ("4s", "4p", "4d"):
            virts.append(ao_idx)


full = []
for f in frags:
    for fi in f:
        full.append(fi)

active = [i for i in full] 
full.extend(doccs)
full.extend(virts)

#
#   Get full active space
(Oact, Sact, Vact), (Oenv, Cerr, Venv) = svd_subspace_partitioning_nonorth((Cdocc, Csing, Cvirt), full, S)
assert(Cerr.shape[1] == 0)

#
#   Get full active space
(Oact, Sact, Vact), (Ocluster, Cerr, Vcluster) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), active, S)
assert(Cerr.shape[1] == 0)


 In svd_subspace_partitioning_nonorth
 Partition  190 orbitals into a total of   49 orbitals
            Index   Sing. Val. Space       
                0   0.99999718            2*
                1   0.99999698            0*
                2   0.99999545            2*
                3   0.99999507            0*
                4   0.99998867            0*
                5   0.99998785            0*
                6   0.99998411            0*
                7   0.99998367            0*
                8   0.99998155            0*
                9   0.99998086            0*
               10   0.99989780            0*
               11   0.99985233            2*
               12   0.99985217            2*
               13   0.99967182            0*
               14   0.99951803            2*
               15   0.99940574            2*
               16   0.99928479            0*
               17   0.99928229            0*
               18   0.99907943            2*
        

In [5]:
#
#   Split active space into fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning_nonorth((Oact, Sact, Vact), f, S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print(" sing vals of active space: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
# for i in range(len(frags)):
#     di = Cfrags[i] @ Cfrags[i].T
#     pyscf.tools.cubegen.density(mf.mol, 'fragden_{:02d}.cube'.format(i+1), di)


# add doccs and virts. We'll add the doccs first, then the virts

init_fspace.append((Ocluster.shape[1], Ocluster.shape[1]))
init_fspace.append((0,0))

tmpi = Cact.shape[1]+1
tmpj = Ocluster.shape[1] 
tmpk = Vcluster.shape[1] 
clusters.append([i for i in range(tmpi,tmpi+tmpj)])
clusters.append([i for i in range(tmpi+tmpj,tmpi+tmpj+tmpk)])

Cact = np.hstack((Cact, Ocluster, Vcluster))


print("init_fspace = ", init_fspace)
print("clusters    = ", clusters)



 Fragment:  [17, 18, 19, 20, 21]
 In svd_subspace_partitioning_nonorth
 Partition   17 orbitals into a total of    5 orbitals
            Index   Sing. Val. Space       
                0   0.92578269            1*
                1   0.92360783            1*
                2   0.91561521            1*
                3   0.90711254            1*
                4   0.89646121            1*
                5   0.21961587            0
                6   0.14292561            0
                7   0.13420994            0
                8   0.06532186            2
                9   0.06247465            2
               10   0.00599335            2
               11   0.00084489            0
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              4              0
              1              5              5
              2              3              0

 Fragment:  [36, 37, 38, 39, 40, 41, 42]
 In svd_subspace_part

# Make integrals

In [6]:
#   Make integrals
#
d1_embed = 2 * Oenv @ Oenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)